In [ ]:
import sys
import pandas as pd
import pickle
sys.path.append('../')

In [ ]:
from api.utils.transformers import *

In [ ]:
from scipy.stats import spearmanr

### Generate Training Data

In [ ]:
generator = TrainingDataGenerator('../../Welcome-Centre-DataCorps-Data/ClientDatabaseStructure.mdb.sqlite')
training_data = generator.get_training_data(limit=100000)

### Build your Transformer and Choose Features

In [ ]:
features_to_split = []

transformer = TransformerPipeline([
            ConsolidateTablesTransformer(),
            AddTimeFeaturesTransformer(),
            SplitCurrentAndEverTransformer(['ReferralIssue_', 'ReferralDomesticCircumstances_',
                                            'ReferralReason_', 'ReferralBenefit_'])
        ], aligner=AlignFeaturesToColumnSchemaTransformer())

X, y, referral_table = transformer.fit_transform(training_data)

X = X.fillna(0)

In [ ]:
X_train = X[X.index < 13500]
X_test = X[X.index >= 13500]
y_train = y.loc[X_train.index]
y_test = y.loc[X_test.index]

# Build a model

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
et = ExtraTreesRegressor(n_jobs=-1, n_estimators=500)
et.fit(X_train, y_train)

# Evaluate model

In [ ]:
y_pred = pd.Series(et.predict(X_test), X_test.index)

In [ ]:
test_referral_table = referral_table.loc[X_test.index]

In [ ]:
def get_scores_per_window(x, y, group, threshold=0.50):
    corr = spearmanr(x, y)[0]
    mu_a = x.groupby(group).mean()
    mu_p = y.groupby(group).mean()
    mu_a_top = mu_a[mu_a.rank(ascending=False) / len(mu_a) < threshold]
    mu_p_top = mu_p[mu_p.rank(ascending=False) / len(mu_p) < threshold]
    overlap = mu_p_top.index.isin(mu_a_top.index).mean()
    return pd.Series([corr, overlap], index=['spearman', 'overlap'])

In [ ]:
def evaluate_average_weekly_rank_correlation(test_referral_table, y_test, y_pred):
    grouped_y = test_referral_table.assign(y=y_test, pred=y_pred).set_index('Referral_ReferralTakenDate')\
        .groupby([pd.TimeGrouper('1W'), 'Client_ClientId'])['y'].mean()
    grouped_pred_y = test_referral_table.assign(y=y_test, pred=y_pred).set_index('Referral_ReferralTakenDate')\
        .groupby([pd.TimeGrouper('1W'), 'Client_ClientId'])['pred'].mean()
    time_grouped = pd.concat([grouped_y, grouped_pred_y], axis=1)
    return time_grouped.reset_index().groupby(['Referral_ReferralTakenDate']).\
        apply(lambda k: get_scores_per_window(k['y'], k['pred'], k['Client_ClientId'])).dropna().mean()

In [ ]:
evaluate_average_weekly_rank_correlation(test_referral_table, y_test, y_pred)

# Save your model

In [ ]:
from api.utils.models import TWCModel

In [ ]:
model = TWCModel()
model.transformer = transformer
model.model = et
model.save('etmodel.p')

In [ ]:
model2 = TWCModel()

In [ ]:
model2.load('etmodel.p')

### Test your model works on json test file

In [ ]:
with open('../api/twc_sample_request.json') as f:
    json_string = f.read()

In [ ]:
p = ParseJSONToTablesTransformer()
example_tables = p.fit_transform(json_string)

In [ ]:
model2.transformer.fit_transform(example_tables)